# Proyecto de análisis de sentimientos con Python


Curso 2022/2023:
    Juan López Quirós
    Jose Ignacio Castro Vázquez

 Lo primero que hay que hacer es escoger o recopilar una primera versión de los datos necesarios que utilizarmos 
 para entrenar a nuestros modelos de Aprendizaje automático. 
 Tras haber estudiado los varios problemas con la API de twitter y las alternativas propuestas, nos decidimos por 
 buscar un dataset ya recopilado de tweets reales de la página web kaggle. En concreto nos decidimos por un dataset
 ya enfocado al análisis de sentimientos con más de 1.6 millones de tweets recopilados directamente de la API de twitter
 por lo que se ajusta perfectamente al proyecto.

 url : https://www.kaggle.com/datasets/kazanova/sentiment140

 Luego, tenemos que limpiar el dataset escogido para este proyecto.
 A nosotros solo nos interesa una columna en particular de todo el dataset y ese es la columna de texto, que contiene
 el contenido de los tweets en sí. Para leer el fichero de datos y procesarlo, utilizaremos pandas.

In [13]:
import pandas as pd

Definimos la función que se encarga de leer y procesar el fichero de datos. Esta función eliminará las columnas irrelevantes, seleccionará un subset de 5000 tweets que formará el corpus de nuestros modelos y por último los guardará en otro fichero para su uso más adelante.

In [14]:
def create_cleaned_csv(original_filename, target_filename):
    #uft-8 encoding didn't work, latin1 encoding did.
    data = pd.read_csv(original_filename, encoding="latin1", header=None)

    #We give the columns a name
    column_names = ["target", "id", "date", "flag", "user", "text"]
    data.columns = column_names

    #We eliminate innecessary columns
    data.drop(columns=["target", "id", "date", "flag", "user"], inplace=True)

    #We select a subset of 5000 tweets
    data = data[:5000]

    #Save data onto new file
    data.to_csv(target_filename, index=False)
    print("FINAL RESULT: \n")
    print(data)

In [15]:
create_cleaned_csv("tweet_data.csv", "subset_tweet_data.csv")        

FINAL RESULT: 

                                                   text
0     @switchfoot http://twitpic.com/2y1zl - Awww, t...
1     is upset that he can't update his Facebook by ...
2     @Kenichan I dived many times for the ball. Man...
3       my whole body feels itchy and like its on fire 
4     @nationwideclass no, it's not behaving at all....
...                                                 ...
4995                                    long day today 
4996                     a friend broke his promises.. 
4997               @gjarnling I am fine thanks - tired 
4998          trying to keep my eyes open..damn baking 
4999                        why the hell is it snowing 

[5000 rows x 1 columns]


A continuación, utilizamos la libreria NLTK para limpiar, tokenizar y lematizar nuestro dataset.

Para esto, descargamos los recursos necesarios para trabajar con la librería nltk.

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jicas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jicas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Para asegurarnos de que el recurso que contiene las stopwords se descargó correctamente, los mostramos por pantalla.

In [17]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{'with', "haven't", 'ma', 'up', "aren't", 'has', 'for', 'any', 'is', 'when', "needn't", 'who', 'how', 'after', 'this', 'shouldn', 'its', 'we', 'why', 'hers', 'by', 'will', 'your', 'did', 'and', 'which', 'was', 'theirs', 'ourselves', 'her', 'now', 'these', 'until', 'where', 'wasn', 's', 'during', 'few', 'doesn', 'about', 'against', 'were', 'more', 'most', 'be', 'my', "should've", 'before', 'that', 'here', 'just', 'haven', 'isn', "wouldn't", "that'll", 'should', 'he', 'while', "isn't", "won't", 'a', 'through', 'me', "shouldn't", 'o', 'their', 'from', 'in', "you'd", "hasn't", "you'll", 'because', 've', 'each', 'above', 'didn', 'his', "you're", "wasn't", 'herself', 'having', 'an', 'some', 'then', 'very', 'doing', 'hasn', 'ain', 'over', 'too', 'needn', 'not', 'shan', 'had', 'our', "doesn't", 'himself', 'out', 'once', 'on', 'same', 'won', "hadn't", 'but', "weren't", "you've", 'yourself', 'at', "it's", 're', 'mightn', 'are', 'd', 'the', 'couldn', 'can', 'don', 'as', 'wouldn', 'myself', 'being

Definimos la función encargada de limpiar los tweets del dataset que eliminará las menciones, hashtags, urls y otros símbolos extraños contenidos en ellos.

Además después de pasar por este filtro, los tokenizamos con TweetTokenizer que a diferencia del tokenizador por defecto de NLTK, word_tokenizer, este si mantiene la coherencia en palabras complejas del inglés.

Por ejemplo la palabra "can't":
- Con word_tokenizer-> can't = ["ca", "n't"]
- Con TweetTokenize-> can't = ["can't"]

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from string import punctuation

def clean_text(original_filename, target_filename, with_punctuation=True):
    data = pd.read_csv(original_filename, encoding="latin1")
    stop_words = set(stopwords.words('english'))
    mention_or_hashtag_characters = ["@", "#"]

    tt = TweetTokenizer() #Tokenizer
    ps = PorterStemmer() #Stemmer

    for i, row in data.iterrows():
        tweet = data.iloc[i]["text"]

        tweet_tokens = tt.tokenize(tweet) #Tokenizing

        filtered_tweet_tokens = [token for token in tweet_tokens if 
                                    token.lower()[0] not in mention_or_hashtag_characters and #Eliminate mentions, hashtags
                                    token.lower() not in stop_words and #Eliminate stop words
                                    'http' not in token.lower()] #Eliminate urls

        if(not with_punctuation):
            filtered_tweet_tokens = [token for token in filtered_tweet_tokens if token not in list(punctuation)]

        stemmed_tweet_tokens = [ps.stem(token) for token in filtered_tweet_tokens] #Stemming
        
        data.loc[i,['text']] = ' '.join(str(token) for token in stemmed_tweet_tokens)

    data.to_csv(target_filename, index=False)

        


Hacemos 2 llamadas a la función. La primera para el corpus sea texto con signos de puntuación inclusive y otro con texto puramente alfanumérico, esto lo hacemos así para estudiar el posible impacto que puedan tener estos en los modelos finales. 

In [19]:
clean_text('subset_tweet_data.csv', 'tokenized_data.csv')
clean_text('subset_tweet_data.csv', 'tokenized_data_no_punctuation.csv', False)

El siguiente paso consisitirá en utilizar la librería TextBlob para la clasificacion de los tweets en: Muy Feliz, Contento, Neutro, Molesto, Hater.

Para ello primero debemos definir los términos y qué consideraremos como 'Neutro', 'Hater', etc..
TextBlob mediante su función 'TextBlob.sentiment' nos hace un análisis de sentimientos de cualquier texto en forma de Objeto Sentiment, un objeto que tiene 2 parámetros:

    - Polarity con rango [-1,1] que indica la polaridad del sentimiento del texto siendo -1 un texto muy negativo y 1 muy positivo
    
    - Subjectivity con rango [0,1] que indica la objetividad del texto siendo 0 muy objetivo y cercano a la realidad y 1 siendo muy subjetivo donde se expresan más las opiniones y sentimientos personales del autor.

Teniendo en cuenta las limitaciones del procesamiento de análsis de Textblob definiremos:

    - Muy Feliz: Polarity=(0.25,1] Subjectivity=[0-0.5)
    - Contento: Polarity=(0.25,1] Subjectivity=[0.5-1]
    - Neutro: Polarity=(-0.25,0.25] Subjectivity=any
    - Hater: Polarity=[-1,-0.25) Subjectivity=[0-0.5)
    - Molesto: Polarity=[-1,-0.25) Subjectivity=[0.5-1]
    
    Esto lo hacemos así ya que consideramos que la principal diferencia entre "Muy Feliz"-"Contento" y "Hater"-"Molesto" es que los primeros son opiniones más subjetivas que las segundas. Y por otra parte consideramos que si el texto es muy objetivo decimos que es neutro ya que no expresa sentimiento ninguno de la persona que escribe el tweet.

In [20]:
from textblob import TextBlob

def classify_tweets(original_filename, target_filename):
    data = pd.read_csv(original_filename, encoding="latin1")
    dic_count = dict()
    for i, row in data.iterrows():
        tweet = TextBlob(str(data.iloc[i]["text"]))
        sentiment = tweet.sentiment
        tag = choose_classification(sentiment)
        data.loc[i,['tag']] = tag

        if tag not in dic_count:
            dic_count[tag] = 1
        else:
            dic_count[tag] = dic_count[tag] + 1

    print(dic_count)
    data.to_csv(target_filename, index=False)


def choose_classification(sentiment):
    classification = "Neutral"
    polarity = sentiment.polarity 
    subjectivity = sentiment.subjectivity 
    if polarity < -0.25:
        classification = "Molesto"
        if subjectivity < 0.5:
            classification = "Hater"
    elif polarity > 0.25:
        classification = "Contento"
        if subjectivity < 0.5:
            classification = "Muy Feliz"
    return classification


Probamos la clasificación de los tweets en los 2 datasets uno con signos de puntuación y otro sin ellos. Podemos ver, por los diccionarios contadores calculados, que devuelven resultados muy parecidos.

In [21]:
classify_tweets("tokenized_data_no_punctuation.csv", "final_dataset.csv")
classify_tweets("tokenized_data.csv", "final_dataset_punctuation.csv")

{'Neutral': 3516, 'Molesto': 656, 'Contento': 596, 'Hater': 80, 'Muy Feliz': 152}
{'Neutral': 3467, 'Molesto': 667, 'Contento': 611, 'Hater': 83, 'Muy Feliz': 172}


Tras la revisión manual de los dataset, el siguiente paso es la implementación de distintos algoritmos de aprendizaje automático. Tras una investigación previa, decidimos implementar estos 3 algoritmos:

    - Naive Bayes
    - Support Vector Machine
    - Convolutional Neural Network

Otros algoritmos que consideramos fueron; Recursive Neural Network, Logistic Regression y modelos Transformers, decidimos descartar ya que....
EXPLICAR POR QUÉ en el documento.

# Seguiremos el siguiente procedimiento para la selección del modelo:

#0: Vectorizar el dataset (tf-itf), countVecotrizer es más simple aunque se puede evaluar cual es mejor.

#1: Dividir dataset en 5 pliegues de igual longitud

#2: Para cada k 1 pliegue sirve de uso para la validación, el resto de entrenamiento. 

#3: Cuando el modelo esté entrenado, evaluar con el conjunto de evaluación y sacar métricas para evaluar rendimiento.

#4: Sacar media de notas de rendimiento del modelo.

#5: REPETIR PARA CADA MODELO A IMPLEMENTAR.


In [22]:
# Naiive Bayes a implementar: Gaussian, Multinomial, Complement, Bernoulli
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def tfidf_vectorize_data(original_filename):
    data = pd.read_csv(original_filename, encoding="latin1")
    X = data['text'].values.astype('U')
    y = data['tag']
    #Vectorize data
    tag_encoder = LabelEncoder()
    vectorizer = TfidfVectorizer(encoding='latin1', ngram_range=(1,2))
    vectorized_X = vectorizer.fit_transform(X)
    #Encode tags
    encoded_y = tag_encoder.fit_transform(y)
    return vectorized_X, encoded_y

x, y = tfidf_vectorize_data("final_dataset.csv")

In [23]:
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import *

multinomial_nb = MultinomialNB()
gaussian_nb = GaussianNB()
complement_nb = ComplementNB()
bernoulli_nb = BernoulliNB()
categorical_nb = CategoricalNB()

nb_models = [multinomial_nb, gaussian_nb, complement_nb, bernoulli_nb, categorical_nb]

X, y = tfidf_vectorize_data("final_dataset.csv")
#cross validate uses k=5 by default   
results = cross_validate(multinomial_nb, X, y, 
                         return_train_score=True)   


In [24]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

 gnb = GaussianNB()

 y_pred = gnb.fit(X_train, y_train).predict(X_test)
 
 print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

# Neural Network

In [25]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (5,5,5,5), max_iter = 1000, solver = 'adam')
mlp.fit(X_train, y_train)

predictions = mlp.predict(X_test)

In [26]:
from sklearn.metrics import classification_report

report = classification_report(y_test,predictions)
print(report)

              precision    recall  f1-score   support

           0       0.16      0.21      0.18       301
           1       0.02      0.76      0.04        41
           2       0.72      0.08      0.14       338
           3       0.00      0.00      0.00        76
           4       0.88      0.34      0.49      1744

    accuracy                           0.28      2500
   macro avg       0.36      0.28      0.17      2500
weighted avg       0.73      0.28      0.38      2500



C:\Users\jicas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jicas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jicas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1344: Unde